In [ ]:
# nse scratchpad initiation
from helper import *
from nse_func import *

# Do assignments
a = assign_var('nse')
for v in a:
    exec(v)

from ib_insync import *
util.startLoop()

# ib =  get_connected('nse', 'live')
ib = IB().connect('127.0.0.1', 3000, clientId=1)

In [ ]:
%%time
# testing dynamic

sell_blks = [] # Initialize sell blocks

# ... make the input dataframes
#______________________________

df_ac = util.df(ib.accountSummary()) # account summary

# ..positions

df_p = util.df(ib.positions())

df_pc = util.df(list(df_p.contract))
df_pc = df_pc[list(df_pc)[:6]]
df_pc = df_pc.rename(columns={'lastTradeDateOrContractMonth': 'expiration'})

df_pc = df_pc.join(df_p[['position', 'avgCost']]) # merge contract with postion and avgCost
df_pos = df_pc.assign(avgCost=np.where(df_pc.secType == 'OPT', df_pc.avgCost, df_pc.avgCost))

# ..fill symbols
if ib.fills():
    df_fc = util.df(list(util.df(ib.fills())['contract'])) # filled contracts
    df_fc = df_fc[list(df_fc)[:6]]
    df_fc = df_fc.rename(columns={'lastTradeDateOrContractMonth': 'expiration'})
    df_fc

    df_fe = util.df(list(util.df(ib.fills())['execution'])) # executed contracts
    df_fe = df_fe[list(df_fe)[4:13]].drop('liquidation', 1) 

    df_fill = df_fc.join(df_fe)

    fill_symbols = list(df_fill.symbol)

else:
    fill_symbols = []

# ..open orders
open_trades = ib.openTrades()

if open_trades:
    df_open=util.df(open_trades)

    # open contracts
    df_oc = util.df(list(df_open['contract']))
    df_oc = df_oc[list(df_oc)[:6]]
    df_oc = df_oc.rename(columns={'lastTradeDateOrContractMonth': 'expiration'})

    # orders
    df_ord = pd.DataFrame([(row.orderId, row.permId, row.action, row.totalQuantity, row.lmtPrice) 
              for idx, row in df_open.order.items()],
            columns = ['orderId', 'permId', 'action', 'totalQuantity', 'lmtPrice'])

    df_openord = df_oc.join(df_ord)

    # filter the existing sells
    df_sx = df_openord[df_openord.action == 'SELL'] 

else:
    df_openord = pd.DataFrame() # Empty open order DataFrame
    df_sx = pd.DataFrame() # Empty SELL open order DataFrame

df_t = targets(ib) # get the targets from opts.pickle, sanitized for remqty

df_buy = df_pos[df_pos.secType == 'OPT'] # filter out any assigned stock

# ... prepare the BUY harvest orders
#_________________________________

if not df_openord.empty:
    df_bx = df_openord[df_openord.action == 'BUY'] # existing buys
    df_buy = df_buy[~df_buy.conId.isin(list(df_bx.conId))]    

# ..Get the buy option prices
buy_opts = ib.qualifyContracts(*[Contract(conId=c) for c in df_buy.conId])
buy_ticks = ib.reqTickers(*buy_opts)
buyOptPrices = {t.contract.conId: t.marketPrice() for t in buy_ticks} # {symbol: undPrice}

df_buy = df_buy.assign(optPrice = df_buy.conId.map(buyOptPrices))
df_buy = df_buy.assign(optPrice = df_buy.optPrice.fillna(prec)) # make NaN to lowest price (prec)

#..get harvest price

# dte
df_buy = df_buy.assign(dte=[get_dte(d) for d in df_buy.expiration])

# first hvstPrice
df_buy = df_buy.assign(hvstPrice=[get_prec(min(hvstPricePct(d)*c,mp*0.9),prec) 
                              for d, c, mp in zip(df_buy.dte, df_buy.avgCost, df_buy.optPrice)])

# adjust to lower hvstPrice for hvstPrice = optPrice
df_buy = df_buy.assign(hvstPrice=np.where((df_buy.optPrice == df_buy.hvstPrice) &
             (df_buy.hvstPrice > prec), df_buy.optPrice - prec, df_buy.hvstPrice))

# make the minimum harvest price to 0.05
df_buy.loc[df_buy.hvstPrice <= 0.0, 'hvstPrice'] = 0.05 

#...prepare to place the orders
buy_orders = [LimitOrder(action='BUY', totalQuantity=-position, lmtPrice=hvstPrice) 
                      for position, hvstPrice in zip(df_buy.position, df_buy.hvstPrice)]

#...prepare to place the orders
buy_orders = [LimitOrder(action='BUY', totalQuantity=-position, lmtPrice=hvstPrice) 
                      for position, hvstPrice in zip(df_buy.position, df_buy.hvstPrice)]

hco = list(zip(buy_opts, buy_orders))

# ... Prepare the SELL sow orders
#________________________________

# .. find out margin breach
margin_breached = False
net_liq = float(df_ac[df_ac.tag == 'NetLiquidation'][:1].value)
init_margin = float(df_ac[df_ac.tag == 'InitMarginReq'][:1].value)

if init_margin >= net_liq*ovallmarginlmt:  # if overall limit is breached
    print("Overall margin limit breached")
    margin_breached = True

if margin_breached == True:

    # cancel all SELL orders
    if not df_openord.empty:
        sell_cancel = [o for x in df_sx.orderId 
           for o in ib.openOrders() 
           if x == o.orderId]

        cancels = [ib.cancelOrder(c) for c in sell_cancel]

else: # overall limit is not breached

    if fill_symbols: # if there are fill symbols

        if not df_sx.empty: # if there are existing SELL orders

            # cancel existing SELLS
            dfsx_cancel = df_sx[df_sx.symbol.isin(fill_symbols)]
            sell_cancel = [o for x in dfsx_cancel.orderId 
               for o in ib.openOrders() 
               if x == o.orderId]

            cancels = [ib.cancelOrder(c) for c in sell_cancel]

        # make target of the cancelled SELLS
        df_sell = df_t[df_t.symbol.isin(fill_symbols)]

    else: # there are no fill symbols

        if df_sx.empty:      # if existing SELL orders are not available
            df_sell = df_t   # all df_targets need to be sold

        else:        # if there openorders exists but is missing some targets
            if not df_openord.empty:
                df_sell = df_t[~df_t.optId.isin(list(df_openord.conId))] # get the targets
            else:
                df_sell = pd.DataFrame([]) # empty dataframe


    # refresh the prices for df_sell
    if not df_sell.empty:
        opts = ib.qualifyContracts(*[Contract(conId=c) for c in df_sell.optId])
        tickers = ib.reqTickers(*opts)
        optPrices = {t.contract.conId: t.marketPrice() for t in tickers} # {symbol: undPrice}
        df_sell = df_sell.assign(optPrice=df_sell.optId.map(optPrices))

        # adjust expPrice to be the max of expPrice and riskyPrice
        df_sell = df_sell.assign(expPrice=pd.DataFrame([df_sell.optId.map(riskyprice(df_sell, prec)), 
                                         df_sell.expPrice]).max())

        # recalculate expRom
        df_sell.assign(expRom=df_sell.expPrice*df_sell.lot/df_sell.undMargin*365/df_sell.dte)

        # prepare the sell blocks
        sell_blks = trade_blocks(ib, df_sell, exchange=exchange)

In [ ]:
df_buy